In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.callbacks import Callback
import datetime

In [2]:
train_dir = './1'
validation_dir = './0'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

Found 3906 images belonging to 2 classes.
Found 896 images belonging to 2 classes.


In [6]:
# 定义CNN模型结构
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
class MyThresholdCallback(Callback):
    """
    自定义回调，用于在达到指定的准确率阈值时停止训练。
    """
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold  # 准确率阈值

    def on_epoch_end(self, epoch, logs=None):
        # 检查验证集的准确率（也可以改为 'accuracy' 检查训练集准确率）
        if logs.get('accuracy') >= self.threshold:
            print(f"\n达到 {self.threshold*100}% 准确率，停止训练！")
            self.model.stop_training = True

In [8]:
# 设置日志目录
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model = build_model()

threshold_callback = MyThresholdCallback(threshold=0.975)

# 设置EarlyStopping回调
# early_stopping_callback = EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)

In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # 根据您的数据集大小可能需要调整
    epochs=15,            # 根据模型的表现可能需要调整
    validation_data=validation_generator,
    validation_steps=50,   # 根据您的数据集大小可能需要调整
    callbacks=[threshold_callback]  # 在训练时使用回调
)

Epoch 1/15
100/100 [==============================] - 58s 581ms/step - loss: 0.6388 - accuracy: 0.7340 - val_loss: 0.3593 - val_accuracy: 0.8348
Epoch 2/15
100/100 [==============================] - 51s 505ms/step - loss: 0.3606 - accuracy: 0.8495
Epoch 3/15
100/100 [==============================] - 51s 515ms/step - loss: 0.3739 - accuracy: 0.8610
Epoch 4/15
100/100 [==============================] - 52s 523ms/step - loss: 0.3364 - accuracy: 0.8615
Epoch 5/15
100/100 [==============================] - 54s 541ms/step - loss: 0.3467 - accuracy: 0.8927
Epoch 6/15
100/100 [==============================] - 51s 514ms/step - loss: 0.2345 - accuracy: 0.9230
Epoch 7/15
100/100 [==============================] - 50s 499ms/step - loss: 0.2119 - accuracy: 0.9204
Epoch 8/15
100/100 [==============================] - 52s 521ms/step - loss: 0.1955 - accuracy: 0.9280
Epoch 9/15
100/100 [==============================] - 52s 517ms/step - loss: 0.1794 - accuracy: 0.9295
Epoch 10/15
100/100 [==========

In [10]:
model.save('parking_lot_occupancy_model_logback.h5')

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 86528)             0

In [12]:
print(f"Run `tensorboard --logdir={log_dir}` to start TensorBoard.")

Run `tensorboard --logdir=logs/fit/20240314-231248` to start TensorBoard.
